In [51]:
import numpy as np
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from logging import raiseExceptions
from tokenize import Double
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import torch
torch.manual_seed(125)
import random
random.seed(125)
import torch_f as torch_f
from modelovae import Node, GRASSEncoder, GRASSDecoder, deserialize
import seaborn as sns
from matplotlib.ticker import MaxNLocator
import pandas as pd


MMD Calculation:

The function calculate_mmd computes the minimum distance between samples from the real and generated datasets and averages these minimum distances to return the MMD.


Coverage Calculation:

The function calculate_coverage counts how many real samples have at least one generated sample within a specified threshold and computes the coverage ratio.


1-NNA Calculation:

The function calculate_1_nna combines both real and generated samples, computes the nearest neighbors, and calculates the accuracy based on how many nearest neighbors match the label of the real samples.

In [52]:
use_gpu = True
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")

In [53]:
def read_tree(filename, dir):
    with open('./' +dir +'/' +filename, "r") as f:
        byte = f.read() 
        return byte

def numerar_nodos(root, count):
    if root is not None:
        numerar_nodos(root.left, count)
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        return 

def count_fn(f):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        return f(*args, **kwargs)
    wrapper.count = 0
    return wrapper

@count_fn
def createNode(data, radius,left = None, right = None):
        """
        Utility function to create a node.
        """
        return Node(data, radius, left, right)

def count_nodes(self):
        """Recursively counts the number of nodes in the tree."""
        count = 1  # Count the current node
        if self.left:
            count += count_nodes(self.left)  # Count left subtree
        if self.right:
            count += count_nodes(self.right)  # Count right subtree
        return count

In [54]:
def decode_testing(v, max, decoder, mult, min):
    def decode_node(v, max, decoder, mult, min):

        cl = decoder.nodeClassifier(v)
        _, label = torch.max(cl, 1)
        label = label.data
        
        
        if label == 1 and createNode.count <= max:

            right, radius = decoder.internalDecoder(v)
                
            d = createNode(1, radius) 
            
            d.right = decode_node(right, max, decoder, mult, min)
            return d

        elif label == 2 and createNode.count <= max:

            left, right, radius = decoder.bifurcationDecoder(v)
                
            d = createNode(1, radius)
            
            d.right = decode_node(right, max, decoder, mult, min)
            d.left = decode_node(left, max, decoder, mult, min)
        
            return d

        elif label == 0 : ##output del classifier
     
            if createNode.count>min:
                #print("mayor que min")
                radio = decoder.featureDecoder(v)
                return createNode(1,radio)
        
            else:
                #print("menor que min")
                right, radius = decoder.internalDecoder(v)
                d = createNode(1, radius) 
                d.right = decode_node(right, max, decoder, mult, min)
                return d

        '''
        elif label == 0 : ##output del classifier
            print("0", createNode.count)
            radio = decoder.featureDecoder(v)
            return createNode(1,radio)  
        '''

    createNode.count = 0
    v = decoder.sample_decoder(v)
    dec = decode_node (v, max, decoder, mult, min)

    return dec

In [55]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
from sklearn.metrics import pairwise_distances


def extract_node_features(tree):
    features = []

    def traverse(node):
        if node is not None:
            # Assuming node.radius is a list or array-like with [x, y, z, r]
            features.append(node.radius.cpu())
            traverse(node.left)
            traverse(node.right)

    traverse(tree)
    return np.array(features)

def calculate_mmd_trees(real_trees, generated_trees):
    # Extract features from each tree
    real_features = [extract_node_features(tree) for tree in real_trees]
    generated_features = [extract_node_features(tree) for tree in generated_trees]
    mmd_values = []
    
    # Iterate over each generated tree
    for g_features in generated_features:
        # Calculate distances from the generated tree to all real trees
        distances = []
        for r_features in real_features:
            # Compute pairwise distances between the current generated tree and a real tree
            g_features = g_features.reshape(-1,4)
            pairwise_dist = pairwise_distances(g_features, r_features)
            # Find the minimum distance to any node in the real tree
            min_distance = np.min(pairwise_dist)
            distances.append(min_distance)
        
        # Take the minimum distance across all real trees
        mmd_values.append(np.min(distances))

    # Average MMD over all generated trees
    mmd_value = np.mean(mmd_values)
    
    return mmd_value


from sklearn.metrics import pairwise_distances

def calculate_coverage(real_trees, generated_trees):
    """
    Calculate the Coverage (COV) between real and generated trees.
    
    Parameters:
    - real_trees: List of original trees (reference set).
    - generated_trees: List of generated trees (target set).
    
    Returns:
    - coverage: Fraction of real trees matched by generated trees.
    """
    # Extract features for real and generated trees
    real_features = [extract_node_features(tree) for tree in real_trees]
    generated_features = [extract_node_features(tree).reshape(-1,4) for tree in generated_trees]

    # Flatten the lists of features
    real_features_flat = np.vstack(real_features)
    generated_features_flat = np.vstack(generated_features)

    # Compute pairwise distances between generated and real trees
    distances = pairwise_distances(generated_features_flat, real_features_flat)

    # For each generated tree, find the nearest real tree
    nearest_neighbors = np.argmin(distances, axis=1)

    # Identify unique real trees that are matched
    matched_real_trees = set(nearest_neighbors)

    # Calculate coverage
    coverage = len(matched_real_trees) / len(real_trees)
    
    return coverage


def calculate_1_nna(real_trees, generated_trees):
    """
    Calculate the 1-Nearest Neighbor Accuracy (1-NNA) between real and generated trees.
    
    Parameters:
    - real_trees: List of original trees (reference set).
    - generated_trees: List of generated trees (target set).
    
    Returns:
    - accuracy: 1-NNA accuracy score.
    """
    # Extract features for real and generated trees
    real_features = [extract_node_features(tree) for tree in real_trees]
    generated_features = [extract_node_features(tree).reshape(-1,4) for tree in generated_trees]

    # Flatten the lists of features
    real_features_flat = np.vstack(real_features)
    generated_features_flat = np.vstack(generated_features)

    # Combine real and generated features
    combined_features = np.vstack((real_features_flat, generated_features_flat))

    # Compute pairwise distances
    distances = pairwise_distances(combined_features)

    # Leave-one-out nearest neighbor classification
    total_correct = 0
    total_samples = len(real_trees) + len(generated_trees)

    # Calculate for generated trees
    for i in range(len(generated_trees)):
        # Remove the current generated tree from the combined set
        remaining = np.delete(combined_features, len(real_trees) + i, axis=0)
        nearest_neighbor_index = np.argmin(distances[len(real_trees) + i, :len(real_trees) + len(generated_trees) - 1])
        if nearest_neighbor_index < len(real_trees):
            total_correct += 1  # Found nearest neighbor in real trees

    # Calculate for real trees
    for i in range(len(real_trees)):
        # Remove the current real tree from the combined set
        remaining = np.delete(combined_features, i, axis=0)
        nearest_neighbor_index = np.argmin(distances[i, :len(real_trees) + len(generated_trees) - 1])
        if nearest_neighbor_index >= len(real_trees):
            total_correct += 1  # Found nearest neighbor in generated trees

    # Calculate accuracy
    accuracy = total_correct / total_samples
    
    return accuracy



In [56]:
def my_collate(batch):
    return batch


class tDataset(Dataset):
    def __init__(self, l, dir, transform=None):
        self.names = l
        self.transform = transform
        self.data = [] #lista con las strings de todos los arboles
        for file in self.names:
            self.data.append(read_tree(file, dir))
        self.trees = []

        for tree in self.data:
            deserial = deserialize(tree)
            self.trees.append(deserial)
            

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        #file = self.names[idx]
        #string = read_tree(file)
        tree = self.trees[idx]
        name = self.names[idx]
        return tree

batch_size = 1

In [57]:
def generate_and_evaluate(real_dataset, n_samples, latent_size, decoder, mult, threshold):
    real_trees = real_dataset.trees
    generated_trees = []
    # Generate synthetic samples
    with torch.no_grad():
        for _ in range(n_samples):
            noise = torch.randn(1, latent_size).to(device)  # Assuming batch size of 1 for each generated sample
            generated_tree = decode_testing(noise, 200, decoder, mult, 20)  # Modify this function as needed
            count = []
            numerar_nodos(generated_tree, count)
            generated_trees.append(generated_tree)
            #print(count_nodes(generated_tree))


    # Calculate metrics
    mmd = calculate_mmd_trees(real_trees, generated_trees)
    coverage = calculate_coverage(real_trees, generated_trees)
    accuracy = calculate_1_nna(real_trees, generated_trees)

    return mmd, coverage, accuracy #, coverage, accuracy

In [58]:
dataset_name = "Intra"  # Replace with your dataset name
p = str(10)
eps = str(0)+str(1)
d = "data/paper/" + dataset_name + "P" +p + "eps" + eps
file_list = os.listdir(d)[:100]
 
#data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)

# Create dataset
real_dataset = tDataset(file_list, d)

# Initialize decoder
a = [1., 1., 1.]
mult = torch.Tensor(a).to(device)
latent_size = 64
Grassdecoder = GRASSDecoder(latent_size=latent_size, hidden_size=256, mult=mult)
Grassdecoder = Grassdecoder.to(device)
Grassdecoder.eval()
checkpoint = torch.load("output/" + dataset_name + "-best.pth")
Grassdecoder.load_state_dict(checkpoint['decoder_state_dict'])


# Generate synthetic trees and evaluate
n_samples = 100
threshold = 1.5  # Adjust as necessary
mmd, coverage, accuracy = generate_and_evaluate(real_dataset, n_samples, latent_size, Grassdecoder, mult, threshold)

# Print results
print(f"Minimum Matching Distance (MMD): {mmd}")
print(f"Coverage (COV): {coverage:.2f}")
print(f"1-Nearest Neighbor Accuracy (1-NNA): {accuracy:.2f}")

C:\Users\User\AppData\Local\Temp\ipykernel_11068\2137478011.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("output/" + dataset_name + "-best.pt

Minimum Matching Distance (MMD): 0.013743375428020954
Coverage (COV): 11.75
1-Nearest Neighbor Accuracy (1-NNA): 0.01


In [59]:
from evaluation_metrics_3d import lgan_mmd_cov

import torch

def _pairwise_EMD_CD_trees(ref_trees, sample_trees, batch_size):
    """
    Compute the pairwise Earth Mover's Distance (EMD) and Chamfer Distance (CD) between
    reference and sample trees.

    Args:
    - ref_trees (list[Node]): List of reference trees (real data).
    - sample_trees (list[Node]): List of sample trees (synthetic data).
    - batch_size (int): Batch size for computations.

    Returns:
    - M_rs_cd (torch.Tensor): Pairwise Chamfer Distance matrix.
    - M_rs_emd (torch.Tensor): Pairwise Earth Mover's Distance matrix.
    """
    num_ref, num_sample = len(ref_trees), len(sample_trees)
    M_rs_cd = torch.zeros((num_ref, num_sample))
    M_rs_emd = torch.zeros((num_ref, num_sample))

    for i, ref_tree in enumerate(ref_trees):
        for j, sample_tree in enumerate(sample_trees):
            # Calculate Chamfer Distance between the reference and sample tree
            M_rs_cd[i, j] = compute_chamfer_distance(ref_tree, sample_tree)

            # Calculate Earth Mover's Distance between the reference and sample tree
            M_rs_emd[i, j] = compute_earth_mover_distance(ref_tree, sample_tree)

    return M_rs_cd, M_rs_emd

def compute_chamfer_distance(tree1, tree2):
    """
    Compute Chamfer Distance (CD) between two trees.

    Args:
    - tree1 (Node): Root node of the first tree.
    - tree2 (Node): Root node of the second tree.

    Returns:
    - cd (float): Chamfer Distance between the two trees.
    """
    nodes1 = extract_tree_nodes(tree1)
    nodes2 = extract_tree_nodes(tree2)

    # Calculate distance from each node in nodes1 to the closest node in nodes2

    dist1 = torch.tensor([min(torch.norm(n1.radius[:3] - n2.radius[0][:3]).item() for n2 in nodes2) for n1 in nodes1])

    # Calculate distance from each node in nodes2 to the closest node in nodes1
    dist2 = torch.tensor([min(torch.norm(n2.radius[0][:3] - n1.radius[:3]).item() for n1 in nodes1) for n2 in nodes2])

    # Chamfer Distance is the mean of these closest distances
    cd = (dist1.mean() + dist2.mean()).item()
    return cd

def compute_earth_mover_distance(tree1, tree2):
    """
    Compute Earth Mover's Distance (EMD) between two trees.

    Args:
    - tree1 (Node): Root node of the first tree.
    - tree2 (Node): Root node of the second tree.

    Returns:
    - emd (float): Earth Mover's Distance between the two trees.
    """
    nodes1 = extract_tree_nodes(tree1)
    nodes2 = extract_tree_nodes(tree2)

    # Calculate EMD as the sum of distances of corresponding nodes
    emd_distances = [torch.norm(n1.radius[:3] - n2.radius[0][:3]).item() for n1, n2 in zip(nodes1, nodes2)]
    emd = sum(emd_distances) / len(emd_distances) if emd_distances else 0.0
    return emd

def extract_tree_nodes(root):
    """
    Extract all nodes from a binary tree using breadth-first traversal.

    Args:
    - root (Node): Root node of the tree.

    Returns:
    - nodes (list[Node]): List of all nodes in the tree.
    """
    nodes = []
    queue = [root]
    while queue:
        current = queue.pop(0)
        nodes.append(current)
        
        # Enqueue left and right children if they exist
        if current.left is not None:
            queue.append(current.left)
        if current.right is not None:
            queue.append(current.right)
    
    return nodes

import torch

def knn_trees(M_real_real, M_real_synth, M_synth_synth, k=1, sqrt=False):
    """
    Computes the 1-Nearest Neighbor accuracy using precomputed pairwise distances.

    Args:
    - M_real_real (torch.Tensor): Pairwise distance matrix between real trees (N_real x N_real).
    - M_real_synth (torch.Tensor): Pairwise distance matrix between real and synthetic trees (N_real x N_synth).
    - M_synth_synth (torch.Tensor): Pairwise distance matrix between synthetic trees (N_synth x N_synth).
    - k (int): Number of neighbors to consider.
    - sqrt (bool): If True, will take the square root of distances to return to original units.

    Returns:
    - results (dict): Dictionary with 1-NN accuracy.
    """
    if sqrt:
        M_real_real = torch.sqrt(M_real_real)
        M_real_synth = torch.sqrt(M_real_synth)
        M_synth_synth = torch.sqrt(M_synth_synth)
    
    # 1-NN accuracy calculation
    N_real = M_real_synth.size(0)
    N_synth = M_real_synth.size(1)
    
    # Concatenate distances between real and synthetic trees
    distances = torch.cat((M_real_real, M_real_synth), dim=1)
    correct = 0

    # For each synthetic tree, find the nearest neighbor among real trees
    for i in range(N_synth):
        nn_index = torch.argmin(M_real_synth[:, i])  # Nearest real tree index for each synthetic tree
        if nn_index < N_real:  # If the nearest neighbor is among real trees
            correct += 1
    
    accuracy = correct / float(N_synth)
    return {"acc": accuracy}


def compute_all_metrics_trees(sample_trees, ref_trees, batch_size, logger):
    """
    Computes Minimum Matching Distance (MMD), Coverage, and 1-Nearest Neighbor Accuracy metrics
    between sample (synthetic) and reference (real) trees.
    
    Args:
    - sample_trees (list[Node]): Generated synthetic trees.
    - ref_trees (list[Node]): Real trees from the dataset.
    - batch_size (int): Batch size for metric computation.
    - logger (tuple): Logger and output directory for saving logs or intermediate results.
    
    Returns:
    - dict: A dictionary containing calculated metrics:
      - "EMD": Minimum Matching Distance and Coverage metrics for EMD.
      - "CD": Minimum Matching Distance and Coverage metrics for CD.
      - "1-NN-CD-acc": 1-Nearest Neighbor Accuracy based on Chamfer Distance.
    """
    results = {}
    logger, orig_trees_dir = logger  # Unpack logger and directory for tree data

    print("Calculating Pairwise EMD and CD distances...")
    # Compute the pairwise Earth Mover's Distance (EMD) and Chamfer Distance (CD)
    M_rs_cd, M_rs_emd = _pairwise_EMD_CD_trees(ref_trees, sample_trees, batch_size)

    # Calculate EMD-based metrics
    res_emd = lgan_mmd_cov(M_rs_emd.t())
    results.update({
        "EMD": res_emd
    })

    # Calculate CD-based metrics
    res_cd = lgan_mmd_cov(M_rs_cd.t())
    results.update({
        "CD": res_cd
    })

    # Print each metric for verification
    for metric, value in results.items():
        print(f"[{metric}] {value}")

    # Calculate 1-Nearest Neighbor (1-NN) Accuracy using Chamfer Distance
    print("Calculating 1-Nearest Neighbor Accuracy (1-NN-CD)...")
    one_nn_cd_res = knn_trees(M_rs_cd, M_rs_cd, M_rs_cd, 1, sqrt=False)
    results.update(
        {"1-NN-CD-acc": one_nn_cd_res["acc"]}  # Assumes "acc" key contains 1-NN accuracy
    )

    return results


In [ ]:
import torch
import os
# Assuming other necessary imports like the GRASSDecoder and any helper functions

def generate_and_evaluate(real_dataset, n_samples, latent_size, decoder, mult, threshold, batch_size, logger):
    # Extract real trees from the dataset
    real_trees = real_dataset.trees  # Adjust based on how tDataset stores trees
    generated_trees = []

    # Generate synthetic trees
    with torch.no_grad():
        for _ in range(n_samples):
            noise = torch.randn(1, latent_size).to(device)  # Batch size of 1 per generated sample
            generated_tree = decode_testing(noise, 200, decoder, mult, 20)  # Generate a synthetic tree
            count = []
            numerar_nodos(generated_tree, count)  # Ensure nodes are correctly numbered
            generated_trees.append(generated_tree)

    # Calculate metrics
    results = compute_all_metrics_trees(generated_trees, real_trees, batch_size, logger)

    # Extract individual metrics from the results dictionary
    mmd = results['EMD']['lgan_mmd']  # Minimum Matching Distance
    coverage = results['EMD']['lgan_cov']  # Coverage
    accuracy = results.get("1-NN-CD-acc", 0.0)  # 1-Nearest Neighbor Accuracy, default to 0 if missing

    return mmd, coverage, accuracy

# Main script for generating and evaluating trees
dataset_name = "Intra"  # Replace with your dataset name
p = str(10)
eps = str(0) + str(1)
d = f"data/paper/{dataset_name}P{p}eps{eps}"
file_list = os.listdir(d)[:100]

# Create dataset
real_dataset = tDataset(file_list, d)

# Initialize decoder
a = [1., 1., 1.]
mult = torch.Tensor(a).to(device)
latent_size = 64
Grassdecoder = GRASSDecoder(latent_size=latent_size, hidden_size=256, mult=mult)
Grassdecoder = Grassdecoder.to(device)
Grassdecoder.eval()

# Load checkpoint
checkpoint = torch.load(f"output/{dataset_name}-best.pth")
Grassdecoder.load_state_dict(checkpoint['decoder_state_dict'])

# Set parameters for generation and evaluation
n_samples = 100
threshold = 1.5  # Adjust if necessary
batch_size = 32  # Set batch size for evaluation
logger = (None, "data/logs/")  # Adjust logger directory if needed

# Generate synthetic trees and calculate metrics
mmd, coverage, accuracy = generate_and_evaluate(real_dataset, n_samples, latent_size, Grassdecoder, mult, threshold, batch_size, logger)

# Print results
print(f"Minimum Matching Distance (MMD): {mmd}")
print(f"Coverage (COV): {coverage:.2f}")
print(f"1-Nearest Neighbor Accuracy (1-NNA): {accuracy:.2f}")


C:\Users\User\AppData\Local\Temp\ipykernel_11068\2940608638.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"output/{dataset_name}-best.pth")


Calculating Pairwise EMD and CD distances...
